In [3]:
from models import CNN
import torch
import torch.nn.functional as F
from torchvision import models, transforms
import numpy as np
import cv2
from PIL import Image
import utils
import Clients
import os
import sys
import DataManager.datamanager as dm
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
import matplotlib.pyplot as plt
from Clients.naive import NaiveClient
from argparse import Namespace
from Clients.base import BaseClient
import torch.nn as nn


In [2]:
import os
print(os.getcwd())
os.chdir("..//")
print(os.getcwd())

c:\Users\user\My Drive\PhD\Projects\YONSEI-TAU FL collaboration\EXPFL


In [5]:
trainset, testset = dm.ECGHeartbeatCategorization()

testloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=False, drop_last=False)
trainloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False, drop_last=False)
for idx, (data, target) in enumerate(testloader):
    if idx == 0:
        input_tensor = data
        break
model = CNN(n_class=5)

In [ ]:

%tb

args = Namespace(exp_name='EXP', model='CNN', dataset='ECG', optimizer='SGD', lr=1e-3, decay=1e-4, batch_size=64, seed=0)
client = BaseClient(args, 'Naive')
client = NaiveClient(args, 'Naive', dm.ECGHeartbeatCategorization)

client.model = utils.build_model(args)
client.model.to(client.device)
client.criterion   = nn.CrossEntropyLoss()
client.optimizer   = torch.optim.Adam(client.model.parameters(), lr=args.lr)
client.trainloader = torch.utils.data.DataLoader(client.trainset, batch_size=args.batch_size, shuffle=True , drop_last=True )
client.testloader  = torch.utils.data.DataLoader(client.testset , batch_size=args.batch_size, shuffle=False, drop_last=False)

for epoch in range(10):
    client.train()
    client.test()
model = client.model

for idx, (data, target) in enumerate(client.testloader):
    if idx == 0:
        input_tensor = data
        break

In [ ]:
cam = GradCAM(model=model, target_layers=[model.conv1])
grayscale_cams = cam(input_tensor=input_tensor)

fig, axes = plt.subplots(3, 10, figsize=(20, 4))
for i in range(3):
    grayscale_cam = grayscale_cams[i, :]
    rgb_img = np.array(data[i].permute(1, 2, 0))

    # Plot figures
    visualization = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)
    axes[0,i].imshow(rgb_img, cmap='gray')
    axes[0,i].set_xticks([])
    axes[0,i].set_yticks([])
    axes[1,i].imshow(grayscale_cam)
    axes[1,i].set_xticks([])
    axes[1,i].set_yticks([])
    axes[2,i].imshow(visualization)
    axes[2,i].set_xticks([])
    axes[2,i].set_yticks([])
plt.tight_layout()
plt.show()